In [1]:
pip install -e . 

Obtaining file:///home/jovyan/data-store/cross-sensor-cal
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cross_sensor_cal (pyproject.toml) ... done
  Created wheel for cross_sensor_cal: filename=cross_sensor_cal-0.1.0-0.editable-py3-none-any.whl size=27465 sha256=c4954653a80cf24abc3d969c44227de9f733b4bd715a0aad5f7adaffc8fe7966
  Stored in directory: /tmp/pip-ephem-wheel-cache-eqtn_v2v/wheels/18/70/06/063f1fc66efdafe44297ee2e506086a9ff3bc04776dd2148a3
Successfully built cross_sensor_cal
  Attempting uninstall: cross_sensor_cal
    Found existing installation: cross_sensor_cal 0.1.0
    Uninstalling cross_sensor_cal-0.1.0:
      Successfully uninstalled cross_sensor_cal-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 154.1 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from src.polygon_extraction import control_function_for_extraction
from pathlib import Path


In [25]:
# Example usage
directory = Path("Reflectance_Uncorrected")  # or wherever your ENVI files live
#polygon_path = Path("/home/jovyan/data-store/polygons/site_polygons.gpkg")

control_function_for_extraction(directory, None)


[DEBUG] Writing to full_extracted_pixels/NEON_D13_NIWO_DP1_L020-1_20230815_reflectance_envi_spectral_data.parquet[DEBUG] Writing to full_extracted_pixels/NEON_D13_NIWO_DP1_L044-1_20230821_reflectance_envi_spectral_data.parquet[DEBUG] Writing to full_extracted_pixels/NEON_D13_NIWO_DP1_L048-1_20230821_reflectance_envi_spectral_data.parquet


[INFO] Processing NEON_D13_NIWO_DP1_L048-1_20230821_reflectance_envi.img with 426 bands as Original_band_


Processing NEON_D13_NIWO_DP1_L048-1_20230821_reflectance_envi.img:   0%|          | 0/63 [00:00<?, ?chunk/s]

[INFO] Processing NEON_D13_NIWO_DP1_L044-1_20230821_reflectance_envi.img with 426 bands as Original_band_


Processing NEON_D13_NIWO_DP1_L044-1_20230821_reflectance_envi.img:   0%|          | 0/75 [00:00<?, ?chunk/s]

[ERROR] Error while processing raster file Reflectance_Uncorrected/NEON_D13_NIWO_DP1_L020-1_20230815_reflectance_envi.img: 'Reflectance_Uncorrected/NEON_D13_NIWO_DP1_L020-1_20230815_reflectance_envi.img' not recognized as a supported file format.


Processing NEON_D13_NIWO_DP1_L048-1_20230821_reflectance_envi.img: 100%|██████████| 63/63 [01:11<00:00,  1.13s/chunk]
Processing NEON_D13_NIWO_DP1_L044-1_20230821_reflectance_envi.img: 100%|██████████| 75/75 [01:20<00:00,  1.08s/chunk]


In [26]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# band names like ENVI_band_417 / Masked_band_12 / Original_band_3
_BAND_RE = re.compile(r"^(ENVI|Masked|Original)_band_(\d+)$")

def _leading_band_columns(colnames):
    """
    Return the leading run of band columns (in order as stored),
    stopping at the first non-band column.
    """
    bands = []
    for c in colnames:
        m = _BAND_RE.match(c)
        if m:
            bands.append(c)
        else:
            break
    return bands

def _band_indices(band_cols):
    idxs = []
    for c in band_cols:
        m = _BAND_RE.match(c)
        if m:
            idxs.append(int(m.group(2)))
    return idxs

def validate_bands_in_dir(
    directory,
    expected_bands: int = 426,
    sample_rows: int = 4000,
    recursive: bool = True,
    value_range: tuple | None = (0.0, 1.0),  # set None to skip; or (0, 10000) if scaled reflectance
    eps_all_zero: float = 1e-12,
    show_preview: bool = True,
    meta_cols = ("Raster_File","CRS","Chunk_Number","Pixel_X","Pixel_Y"),
):
    """
    For each Parquet file in `directory`:
      - Identify leading band columns (assumes bands come first).
      - Check band count == expected_bands.
      - Sample values to test: finite, not-all-zero per band, and optional range.
      - Print a compact summary and return a DataFrame with results.
    """
    directory = Path(directory)
    files = sorted(directory.rglob("*.parquet") if recursive else directory.glob("*.parquet"))
    if not files:
        print(f"[INFO] No .parquet files in {directory}")
        return pd.DataFrame()

    pd.set_option("display.max_colwidth", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_columns", None)

    rows = []
    for f in files:
        rec = {
            "file": str(f),
            "ok_open": False,
            "num_row_groups": None,
            "num_rows_meta": None,
            "band_count": None,
            "first_cols_head": None,
            "first_band": None,
            "last_band": None,
            "band_index_min": None,
            "band_index_max": None,
            "bands_ok": None,
            "finite_ok": None,
            "all_zero_bands": [],
            "range_fail_bands": [],
            "error": None,
        }
        try:
            pf = pq.ParquetFile(f)
            rec["ok_open"] = True
            rec["num_row_groups"] = pf.num_row_groups
            rec["num_rows_meta"] = pf.metadata.num_rows if pf.metadata else None

            schema_cols = pf.schema_arrow.names
            rec["first_cols_head"] = schema_cols[:8]

            # take bands as the LEADING run
            band_cols = _leading_band_columns(schema_cols)
            rec["band_count"] = len(band_cols)

            # compute band index stats
            idxs = _band_indices(band_cols)
            if idxs:
                rec["band_index_min"] = int(np.min(idxs))
                rec["band_index_max"] = int(np.max(idxs))
            rec["first_band"] = band_cols[0] if band_cols else None
            rec["last_band"]  = band_cols[-1] if band_cols else None

            # expectation
            rec["bands_ok"] = (rec["band_count"] == expected_bands)

            # Read a small sample from the first row group (fast) for value checks
            cols_to_read = band_cols + [c for c in meta_cols if c in schema_cols]
            table = pf.read_row_group(0, columns=cols_to_read)
            df = table.to_pandas()
            if len(df) > sample_rows:
                df = df.head(sample_rows)
            band_df = df[band_cols].astype("float32") if band_cols else pd.DataFrame()

            # finite check
            if not band_df.empty:
                finite_ok = np.isfinite(band_df.to_numpy()).all()
            else:
                finite_ok = False
            rec["finite_ok"] = bool(finite_ok)

            # not all zero per band
            all_zero_bands = []
            if not band_df.empty:
                # max abs per band
                max_abs = band_df.abs().max(axis=0)
                for c, m in max_abs.items():
                    if not (m > eps_all_zero):
                        all_zero_bands.append(c)
            rec["all_zero_bands"] = all_zero_bands

            # range check (optional)
            range_fail_bands = []
            if value_range is not None and not band_df.empty:
                lo, hi = value_range
                # check band-wise if ANY sampled value is out of range
                too_low  = (band_df < lo).any(axis=0)
                too_high = (band_df > hi).any(axis=0)
                for c in band_cols:
                    if (c in too_low.index and bool(too_low[c])) or (c in too_high.index and bool(too_high[c])):
                        range_fail_bands.append(c)
            rec["range_fail_bands"] = range_fail_bands

            # Optional tiny preview for human eyes
            if show_preview:
                print(f"\n=== {f.name} ===")
                print(f"leading band cols: {rec['band_count']} (expected {expected_bands}) | "
                      f"first: {rec['first_band']} | last: {rec['last_band']} | "
                      f"indices: [{rec['band_index_min']}, {rec['band_index_max']}] | "
                      f"row_groups: {rec['num_row_groups']} | rows(meta): {rec['num_rows_meta']}")
                if not df.empty:
                    preview_cols = [c for c in meta_cols if c in df.columns] + band_cols[:5]
                    display(df[preview_cols].head(1))

        except Exception as e:
            rec["error"] = str(e)

        rows.append(rec)

    out = pd.DataFrame(rows)

    # Directory-level summary
    print("\n===== BAND LAYOUT VALIDATION =====")
    print(f"Directory: {directory}")
    print(f"Files: {len(out)}")
    print(f"Open errors: {(out['ok_open'] == False).sum()}")
    print(f"Band-count mismatches: {(out['bands_ok'] == False).sum()} (expected {expected_bands})")
    print(f"Finite check failures: {(out['finite_ok'] == False).sum()}")
    print(f"Files with all-zero bands (any): {(out['all_zero_bands'].apply(lambda x: len(x) > 0)).sum()}")
    if value_range is not None:
        print(f"Range check failures (any band outside {value_range}): {(out['range_fail_bands'].apply(lambda x: len(x) > 0)).sum()}")

    # Compact view
    compact = out[[
        "file","band_count","bands_ok","first_band","last_band",
        "band_index_min","band_index_max","num_row_groups","num_rows_meta",
        "finite_ok","all_zero_bands","range_fail_bands","error"
    ]].copy()
    display(compact.head(50))
    return out

# For reflectance 0–1:
_ = validate_bands_in_dir(
    "full_extracted_pixels",
    expected_bands=426,
    value_range=(0.0, 10000.0),
    sample_rows=4000,
    recursive=True,
    show_preview=True
)

# If your reflectance is scaled (e.g., 0–10000), change the range:
# _ = validate_bands_in_dir("/home/jovyan/data-store/output_parquet", expected_bands=426, value_range=(0, 10000))



=== NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet ===
leading band cols: 426 (expected 426) | first: ENVI_band_1 | last: ENVI_band_426 | indices: [1, 426] | row_groups: 146 | rows(meta): 7925632


,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4,ENVI_band_5
0,NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_envi.img,EPSG:32613,0,453420.0,4437541.0,169.0,216.0,176.0,196.0,142.72525



=== NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet ===
leading band cols: 426 (expected 426) | first: ENVI_band_1 | last: ENVI_band_426 | indices: [1, 426] | row_groups: 181 | rows(meta): 11151207


,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4,ENVI_band_5
0,NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi.img,EPSG:32613,0,453780.0,4436611.0,78.0,80.0,118.0,69.0,56.265697



=== NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet ===
leading band cols: 426 (expected 426) | first: ENVI_band_1 | last: ENVI_band_426 | indices: [1, 426] | row_groups: 146 | rows(meta): 7925632


...

,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4
0,NEON_D13_NIWO_DP1_L048-1_20230821_resampled_MicaSense-to-match_TM_and_ETM+_envi.img,EPSG:32613,0,445783.0,4437524.0,878.343384,1253.131226,1419.592407,2184.144287



=== NEON_D13_NIWO_DP1_L048-1_20230821_resampled_MicaSense_envi_spectral_data.parquet ===
leading band cols: 10 (expected 426) | first: ENVI_band_1 | last: ENVI_band_10 | indices: [1, 10] | row_groups: 63 | rows(meta): 4407991


,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4,ENVI_band_5
0,NEON_D13_NIWO_DP1_L048-1_20230821_resampled_MicaSense_envi.img,EPSG:32613,0,445783.0,4437524.0,778.666199,878.343384,1099.335571,1253.131226,1449.075317



===== BAND LAYOUT VALIDATION =====
Directory: full_extracted_pixels
Files: 44
Open errors: 0
Band-count mismatches: 35 (expected 426)
Finite check failures: 0
Files with all-zero bands (any): 2
Range check failures (any band outside (0.0, 10000.0)): 36


,file,band_count,bands_ok,first_band,last_band,band_index_min,band_index_max,num_row_groups,num_rows_meta,finite_ok,all_zero_bands,range_fail_bands,error
0,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet,426,True,ENVI_band_1,ENVI_band_426,1,426,146,7925632,True,[],"[ENVI_band_17, ENVI_band_18, ENVI_band_19, ENVI_band_20, ENVI_band_21, ENVI_band_22, ENVI_band_23, ENVI_band_24, ENVI_band_25, ENVI_band_26, ENVI_band_27, ENVI_band_28, ENVI_band_29, ENVI_band_30, ENVI_band_31, ENVI_band_32, ENVI_band_33, ENVI_band_34, ENVI_band_35, ENVI_band_36, ENVI_band_37, ENVI_band_38, ENVI_band_39, ENVI_band_40, ENVI_band_41, ENVI_band_42, ENVI_band_43, ENVI_band_44, ENVI_band_45, ENVI_band_46, ENVI_band_47, ENVI_band_48, ENVI_band_49, ENVI_band_50, ENVI_band_51, ENVI_band_52, ENVI_band_53, ENVI_band_54, ENVI_band_55, ENVI_band_56, ENVI_band_57, ENVI_band_58, ENVI_band_59, ENVI_band_61, ENVI_band_62, ENVI_band_63, ENVI_band_64, ENVI_band_65, ENVI_band_66, ENVI_band_67, ENVI_band_68, ENVI_band_69]",None
1,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet,426,True,ENVI_band_1,ENVI_band_426,1,426,181,11151207,True,[],"[ENVI_band_58, ENVI_band_61]",None
2,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet,426,True,ENVI_band_1,ENVI_band_426,1,426,146,7925632,True,[],"[ENVI_band_17, ENVI_band_18, ENVI_band_19, ENVI_band_20, ENVI_band_21, ENVI_band_22, ENVI_band_23, ENVI_band_24, ENVI_band_25, ENVI_band_26, ENVI_band_27, ENVI_band_28, ENVI_band_29, ENVI_band_30, ENVI_band_31, ENVI_band_32, ENVI_band_33, ENVI_band_34, ENVI_band_35, ENVI_band_36, ENVI_band_37, ENVI_band_38, ENVI_band_39, ENVI_band_40, ENVI_band_41, ENVI_band_42, ENVI_band_43, ENVI_band_44, ENVI_band_45, ENVI_band_46, ENVI_band_47, ENVI_band_48, ENVI_band_49, ENVI_band_50, ENVI_band_51, ENVI_band_52, ENVI_band_53, ENVI_band_54, ENVI_band_55, ENVI_band_56, ENVI_band_57, ENVI_band_58, ENVI_band_59, ENVI_band_61, ENVI_band_62, ENVI_band_63, ENVI_band_64, ENVI_band_65, ENVI_band_66, ENVI_band_67, ENVI_band_68, ENVI_band_69]",None
3,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_5_TM_envi_spectral_data.parquet,6,False,ENVI_band_1,ENVI_band_6,1,6,146,7925632,True,[],"[ENVI_band_1, ENVI_band_2, ENVI_band_3]",None
4,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_7_ETM+_envi_spectral_data.parquet,6,False,ENVI_band_1,ENVI_band_6,1,6,146,7925632,True,[],"[ENVI_band_1, ENVI_band_2, ENVI_band_3]",None
5,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_8_OLI_envi_spectral_data.parquet,7,False,ENVI_band_1,ENVI_band_7,1,7,146,7925632,True,[],"[ENVI_band_2, ENVI_band_3, ENVI_band_4]",None
6,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_9_OLI-2_envi_spectral_data.parquet,7,False,ENVI_band_1,ENVI_band_7,1,7,146,7925632,True,[],"[ENVI_band_2, ENVI_band_3, ENVI_band_4]",None
7,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_MicaSense-to-match_OLI_and_OLI-2_envi_spectral_data.parquet,5,False,ENVI_band_1,ENVI_band_5,1,5,146,7925632,True,[],"[ENVI_band_1, ENVI_band_2, ENVI_band_3, ENVI_band_4]",None
8,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_MicaSense-to-match_TM_and_ETM+_envi_spectral_data.parquet,4,False,ENVI_band_1,ENVI_band_4,1,4,146,7925632,True,[],"[ENVI_band_1, ENVI_band_2, ENVI_band_3]",None
9,full_extracted_pixels/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_MicaSense_envi_spectral_data.parquet,10,False,ENVI_band_1,ENVI_band_10,1,10,146,7925632,True,[],"[ENVI_band_1, ENVI_band_2, ENVI_band_3, ENVI_band_4, ENVI_band_5, ENVI_band_6, ENVI_band_7, ENVI_band_8]",None


In [15]:
# Re-define without importing pyarrow at module import time (to avoid errors in this sandbox).
# We'll import pyarrow inside the function at call-time in the user's environment.

from pathlib import Path
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

_BAND_RE = re.compile(r"^(ENVI|Masked|Original)_band_(\d+)$")

def _leading_band_columns_sorted(colnames):
    bands = []
    for c in colnames:
        m = _BAND_RE.match(c)
        if m:
            bands.append((c, int(m.group(2))))
        else:
            break
    bands.sort(key=lambda x: x[1])
    return [c for c, _ in bands], [i for _, i in bands]

def plot_spectra_from_parquet_dir(
    directory,
    output_dir: str | None = None,
    recursive: bool = True,
    max_lines_per_file: int = 5000,
    median_sample_rows: int = 50000,
    sentinel_at_or_below: float = -9990,
    mask_integer_lo: int = 1,
    mask_integer_hi: int = 255,
    dpi: int = 200
):
    """
    For each Parquet file in `directory`, create a PNG plot:
      - X-axis: band index
      - Y-axis: reflectance
      - One line per pixel (subsampled to at most `max_lines_per_file`)
      - A median (50th percentile) trendline across a uniform sample of pixels

    Notes:
      - Imports `pyarrow.parquet` at call-time. Ensure `pyarrow` is installed in your environment.
    """
    try:
        import pyarrow.parquet as pq  # imported here to avoid environment issues in this sandbox
    except Exception as e:
        raise ImportError("This function requires 'pyarrow'. Please install it (e.g., `pip install pyarrow`).") from e

    directory = Path(directory)
    out_dir = Path(output_dir) if output_dir else (directory / "_spectral_plots")
    out_dir.mkdir(parents=True, exist_ok=True)

    files = sorted(directory.rglob("*.parquet") if recursive else directory.glob("*.parquet"))
    if not files:
        print(f"[INFO] No .parquet files found under: {directory}")
        return []

    generated = []
    rng = np.random.default_rng(42)

    for f in files:
        try:
            pf = pq.ParquetFile(f)
        except Exception as e:
            print(f"[WARN] Skipping {f.name}: cannot open ({e})")
            continue

        cols = pf.schema_arrow.names
        band_cols, band_idx = _leading_band_columns_sorted(cols)
        if not band_cols:
            print(f"[WARN] {f.name}: no leading band columns found; skipping.")
            continue

        total_rows = pf.metadata.num_rows if pf.metadata and pf.metadata.num_rows is not None else None
        if total_rows is None or total_rows == 0:
            print(f"[WARN] {f.name}: no rows; skipping.")
            continue
        stride = max(1, math.ceil(total_rows / max_lines_per_file)) if max_lines_per_file else 1
        p = min(1.0, median_sample_rows / total_rows) if median_sample_rows else 1.0

        fig = plt.figure(figsize=(10, 6))
        ax = plt.gca()

        x = np.array(band_idx, dtype=np.float32)

        samples = []
        plotted = 0
        processed_rows = 0

        for rg in range(pf.num_row_groups):
            try:
                table = pf.read_row_group(rg, columns=band_cols)
            except Exception as e:
                print(f"[WARN] {f.name}: failed to read row group {rg} ({e}); continuing.")
                continue

            df = table.to_pandas()
            arr = df.to_numpy(dtype=np.float32, copy=False)

            # Missing-value masks
            arr = np.where(arr <= sentinel_at_or_below, np.nan, arr)
            int_mask = (arr >= mask_integer_lo) & (arr <= mask_integer_hi) & (arr == np.floor(arr))
            arr = np.where(int_mask, np.nan, arr)

            rows_in_rg = arr.shape[0]

            if max_lines_per_file:
                idxs = np.arange(rows_in_rg)
                plot_mask = ((idxs + processed_rows) % stride == 0)
                plot_rows = arr[plot_mask]
                for row in plot_rows:
                    if np.all(np.isnan(row)):
                        continue
                    ax.plot(x, row, linewidth=0.5, alpha=0.15)  # no explicit color
                    plotted += 1

            if median_sample_rows:
                if p >= 1.0:
                    take = arr
                else:
                    k = int(np.ceil(rows_in_rg * p))
                    if k > 0 and rows_in_rg > 0:
                        sel = rng.choice(rows_in_rg, size=min(k, rows_in_rg), replace=False)
                        take = arr[sel]
                    else:
                        take = None
                if take is not None and take.size > 0:
                    samples.append(take)

            processed_rows += rows_in_rg

        if median_sample_rows and samples:
            sample_arr = np.vstack(samples)
            if sample_arr.shape[0] > median_sample_rows:
                sel = rng.choice(sample_arr.shape[0], size=median_sample_rows, replace=False)
                sample_arr = sample_arr[sel]
            med = np.nanmedian(sample_arr, axis=0)
            ax.plot(x, med, linewidth=2.0, alpha=1.0, label="Median (50th)")
            ax.legend()

        ax.set_xlabel("Band index")
        ax.set_ylabel("Reflectance")
        ax.set_title(f.name)
        fig.tight_layout()

        out_path = out_dir / f"{f.stem}_spectra.png"
        try:
            fig.savefig(out_path, dpi=200, bbox_inches="tight")
            generated.append(out_path)
        except Exception as e:
            print(f"[WARN] Failed to save plot for {f.name}: {e}")
        finally:
            plt.close(fig)

        print(f"[OK] {f.name}: plotted ~{plotted} pixel lines -> {out_path}")

    return generated

# Example run (adjust the path)
plots = plot_spectra_from_parquet_dir("full_extracted_pixels")

# Or customize:
# plots = plot_spectra_from_parquet_dir(
#     "/home/jovyan/data-store/output_parquet",
#     output_dir="/home/jovyan/data-store/output_plots",
#     max_lines_per_file=8000,        # more/less pixel lines on each plot
#     median_sample_rows=100000,      # sample size for the 50th-percentile trend line
#     sentinel_at_or_below=-9990,     # treat <= -9990 (e.g., -9999) as missing
#     mask_integer_lo=1, mask_integer_hi=255  # palette-like integers treated as missing
# )


plot_spectra_from_parquet_dir is ready. Example:
plot_spectra_from_parquet_dir('/home/jovyan/data-store/output_parquet')


In [ ]:
# Example run (adjust the path)
plots = plot_spectra_from_parquet_dir("full_extracted_pixels")

# Or customize:
# plots = plot_spectra_from_parquet_dir(
#     "/home/jovyan/data-store/output_parquet",
#     output_dir="/home/jovyan/data-store/output_plots",
#     max_lines_per_file=8000,        # more/less pixel lines on each plot
#     median_sample_rows=100000,      # sample size for the 50th-percentile trend line
#     sentinel_at_or_below=-9990,     # treat <= -9990 (e.g., -9999) as missing
#     mask_integer_lo=1, mask_integer_hi=255  # palette-like integers treated as missing
# )


[OK] NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet: plotted ~4998 pixel lines -> full_extracted_pixels/_spectral_plots/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data_spectra.png
[OK] NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet: plotted ~4999 pixel lines -> full_extracted_pixels/_spectral_plots/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data_spectra.png
[OK] NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet: plotted ~4998 pixel lines -> full_extracted_pixels/_spectral_plots/NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi_spectral_data_spectra.png
[OK] NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_5_TM_envi_spectral_data.parquet: plotted ~4998 pixel l